In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize_2022 import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt
from itertools import permutations, product
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

strategy3 = True

In [3]:
dataObject = DataProcess()

# static measurement jacobian
if strategy3: 
    static_ind = [0,1,2,3,4,5,6,7,8,9,10]
else:
    static_ind = [0,1,2,4,5,6,8,9,10]
static_Nt = 22
static_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(static_Nt)+'_scale.csv')
num_static = len(static_ind)

static_Q = []
for i in static_ind:
    static_Q.append(dataObject.split_jaco(static_jaco, i,static_Nt))

    

dynamic_ind = [3,7,11,12,13]
    
dynamic_Nt = 22
num_dynamic = len(dynamic_ind)
dynamic_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(dynamic_Nt)+'_scale.csv')

dynamic_Q = []
for i in dynamic_ind:
    dynamic_Q.extend(dataObject.split_jaco(dynamic_jaco, i, dynamic_Nt))
    
print("static Q shape:", np.shape(static_Q))
print("dynamic Q shape:", np.shape(dynamic_Q))

jacobian shape: (308, 5)
jacobian shape: (308, 5)
static Q shape: (11, 22, 5)
dynamic Q shape: (110, 5)


In [4]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]

num_param = 5
cost = None

num_total = num_static + num_dynamic*dynamic_Nt
print(num_total)

121


In [5]:
calculator = MeasurementOptimizer(static_Q, dynamic_Q, static_Nt, dynamic_Nt, num_param, cost, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(14641, 5, 5)


In [23]:
def check(fim_set):
    for i in range(num_total):
        for j in range(num_total):
            fim = fim_set[i*num_total+j]
            if np.any(fim):
                print("unit FIM: ", i, j)
                

In [24]:
check(calculator.fim_collection)

unit FIM:  0 0
unit FIM:  0 1
unit FIM:  0 2
unit FIM:  0 3
unit FIM:  0 4
unit FIM:  0 5
unit FIM:  0 6
unit FIM:  0 7
unit FIM:  0 8
unit FIM:  0 9
unit FIM:  0 10
unit FIM:  1 0
unit FIM:  1 1
unit FIM:  1 2
unit FIM:  1 3
unit FIM:  1 4
unit FIM:  1 5
unit FIM:  1 6
unit FIM:  1 7
unit FIM:  1 8
unit FIM:  1 9
unit FIM:  1 10
unit FIM:  2 0
unit FIM:  2 1
unit FIM:  2 2
unit FIM:  2 3
unit FIM:  2 4
unit FIM:  2 5
unit FIM:  2 6
unit FIM:  2 7
unit FIM:  2 8
unit FIM:  2 9
unit FIM:  2 10
unit FIM:  3 0
unit FIM:  3 1
unit FIM:  3 2
unit FIM:  3 3
unit FIM:  3 4
unit FIM:  3 5
unit FIM:  3 6
unit FIM:  3 7
unit FIM:  3 8
unit FIM:  3 9
unit FIM:  3 10
unit FIM:  4 0
unit FIM:  4 1
unit FIM:  4 2
unit FIM:  4 3
unit FIM:  4 4
unit FIM:  4 5
unit FIM:  4 6
unit FIM:  4 7
unit FIM:  4 8
unit FIM:  4 9
unit FIM:  4 10
unit FIM:  5 0
unit FIM:  5 1
unit FIM:  5 2
unit FIM:  5 3
unit FIM:  5 4
unit FIM:  5 5
unit FIM:  5 6
unit FIM:  5 7
unit FIM:  5 8
unit FIM:  5 9
unit FIM:  5 10
unit

In [32]:
print(calculator.fim_collection[15])

[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [34]:
def check_fim(idx):
    fim = np.asarray(calculator.Q[idx]).T@np.asarray(calculator.Q[idx])
    print(fim)
    print("det:", np.linalg.det(fim))
    print("trace:", np.trace(fim))
    
check_fim(10)
print(calculator.total_no_measure)

[[ 2.9863945   0.04871775  1.07400206 -0.80548264 -4.27287343]
 [ 0.04871775  0.00971182 -0.01154336  0.19094565 -0.07980516]
 [ 1.07400206 -0.01154336  0.73633032 -0.87045855 -1.74208677]
 [-0.80548264  0.19094565 -0.87045855  5.40430152  0.78646997]
 [-4.27287343 -0.07980516 -1.74208677  0.78646997  6.35883207]]
det: 1.6413645530679245e-05
trace: 15.49557023589443
121


### Set up grey-box 

In [7]:
from greybox_generalize import LogDetModel

In [8]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [9]:
if strategy3: 
    cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        7000, #ads.z_out (3)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
        7000, #des.z_out (7) 
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]
else:
    cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]

for i in range(dynamic_Nt*num_dynamic):
    cost.append(100)

install_cost_dynamic = [100, 100, 500, 500, 500]


In [10]:
init_y = np.random.rand(num_total-num_static)

update_y = [1]*num_static 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [11]:
def compute_FIM(numRes, FIMSet, cost_list, mixed_integer=False,  obj="A", 
                num_fixed = 9, num_dynamic=5, num_dynamic_t = 100,
                num_dynamic_t_name = None, discretize_time = None, end_time=220,
                fix=False, sparse=False, manual_number=20, budget=100,
               dynamic_install_cost = None, each_manual_number=None):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    
    # dynamic measurements num 
    m.DimDynamic = pyo.Set(initialize=range(num_fixed, num_dynamic+num_fixed))
    m.DimDynamic_t = pyo.Set(initialize=range(num_dynamic_t))
    
    dynamic_time = {}
    for i in range(num_dynamic_t):
        dynamic_time[i] = num_dynamic_t_name[i]
    
    # set up y and cov y 
    def identity(m, a, b):
        return 1 if a==b else 0
    
    def initialize(m, a, b):
        return init_cov_y[a][b]
    
    def NumReshalf_init(m):
        return ((a,b) for a in m.NumRes for b in range(a, numRes))
    
    def DimFIMhalf_init(m):
        return ((a,b) for a in m.DimFIM for b in range(a, len(FIMSet[0])))
    
    m.NumRes_half = pyo.Set(dimen=2, initialize=NumReshalf_init)
    m.DimFIM_half = pyo.Set(dimen=2, initialize=DimFIMhalf_init)

    if mixed_integer:
        if sparse:
            m.cov_y = pyo.Var(m.NumRes_half, initialize=initialize, bounds=(0,1), within=pyo.Binary)
        else:
            m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.Binary)
    else:
        if sparse:
            m.cov_y = pyo.Var(m.NumRes_half, initialize=initialize, bounds=(1E-6,1), within=pyo.NonNegativeReals)
        else:
            m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
        
    def init_fim(m, p, q):
        return fim_prior[p,q]   
    
    # set up FIM
    if sparse:
        m.TotalFIM = pyo.Var(m.DimFIM_half, initialize=init_fim)
    else:
        m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=init_fim)

    # compute FIM
    def eval_fim(m, a, b):
        if a <= b: 
            summi = 0 
            for i in range(numRes):
                for j in range(numRes):
                    if j>=i:
                        summi += m.cov_y[i,j]*FIMSet[i*numRes+j][a][b]
                    else:
                        summi += m.cov_y[j,i]*FIMSet[i*numRes+j][a][b]
            return m.TotalFIM[a,b] == summi
        else:
            return m.TotalFIM[a,b] == m.TotalFIM[b,a]
    
    m.TotalDynamic = pyo.Var(initialize=1)
    
    def total_dynamic(m):
        return m.TotalDynamic==sum(m.cov_y[i,i] for i in range(num_fixed, numRes))
    
    m.manual = pyo.Constraint(rule=total_dynamic)
    #m.manual = pyo.Expression(expr=sum(m.cov_y[i,i] for i in range(num_fixed, numRes)))
            
    # y and conv y constraints
    def y_covy1(m, a, b):
        if b > a:
            return m.cov_y[a, b] <= m.cov_y[a, a]
        else:
            return pyo.Constraint.Skip
        
    def y_covy2(m, a, b):
        if b > a:
            return m.cov_y[a, b] <= m.cov_y[b, b]
        else:
            return pyo.Constraint.Skip
        
    def y_covy3(m, a, b):
        if b>a:
            return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
        else:
            return pyo.Constraint.Skip
        
    def y_covy4(m, a, b):
        if a>b:
            return m.cov_y[a, b] == m.cov_y[a, a]*m.cov_y[b,b]
        else:
            return pyo.Constraint.Skip
        
    def symmetry(m,a,b):
        if a>b:
            return m.cov_y[a,b] == m.cov_y[b,a]
        else:
            return pyo.Constraint.Skip
    
    def cost_compute(m):
        cost = sum(m.cov_y[i,i]*cost_list[i] for i in range(numRes))+sum(m.if_install_dynamic[j]*dynamic_install_cost[j-num_fixed] for j in m.DimDynamic)
        return m.cost == cost

    def cost_limit(m):
        return m.cost <= budget
    
    def total_dynamic_con(m):
        return m.TotalDynamic<=manual_number
    
    def dynamic_fix_yd(m,i,j):
        # map measurement index i to its dynamic_flatten index
        start = num_fixed + (i-num_fixed)*num_dynamic_t+j
        return m.if_install_dynamic[i] >= m.cov_y[j,j]
    
    def dynamic_fix_yd_con2(m,i):
        start = num_fixed + (i-num_fixed)*num_dynamic_t
        end = num_fixed + (i-num_fixed+1)*num_dynamic_t
        return m.if_install_dynamic[i] <= sum(m.cov_y[j,j] for j in range(start, end))
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.TotalFIM[j,j] for j in m.DimFIM)
        return sum_x
    
    
    # add constraints
    if sparse:
        m.TotalFIM_con = pyo.Constraint(m.DimFIM_half, rule=eval_fim)
    else:
        m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        
        if mixed_integer and not sparse:
            m.sym = pyo.Constraint(m.NumRes, m.NumRes, rule=symmetry)
        
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
            
        m.con_manual = pyo.Constraint(rule=total_dynamic_con)
        
        # each manual number smaller than 5 
        if each_manual_number:
            
            for i in range(num_dynamic):
                def dynamic_manual_num(m):
                    start = num_fixed + i*num_dynamic_t
                    end = num_fixed + (i+1)*num_dynamic_t
                    cost = sum(m.cov_y[j,j] for j in range(start, end))
                    return cost <= each_manual_number
                
                con_name = "con"+str(i)
                m.add_component(con_name, pyo.Constraint(expr=dynamic_manual_num))
                
        if discretize_time:
            
            for i in m.DimDynamic:
                for t in range(num_dynamic_t):
                    # end time is an open end of the region, so another constraint needs to be added to include end_time
                    #if dynamic_time[t]+discretize_time <= end_time+0.1*discretize_time:       
                                              
                    def discretizer(m):
                        sumi = 0

                        count = 0 
                        while (count+t<num_dynamic_t) and (dynamic_time[count+t]-dynamic_time[t])<discretize_time:
                            surro_idx = num_fixed + (i-num_fixed)*num_dynamic_t + t + count
                            sumi += m.cov_y[surro_idx, surro_idx]
                            count += 1 

                        return sumi <= 1 

                        #return sum(cov_y[j,j] for j in disc_ind) <= 1 

                    con_name="con_discreti_"+str(i)+str(t)
                    m.add_component(con_name, pyo.Constraint(expr=discretizer))
                    
        # dynamic-cost measurements installaction cost 
        m.if_install_dynamic = pyo.Var(m.DimDynamic, initialize=0, bounds=(0,1.01))
        m.dynamic_cost = pyo.Constraint(m.DimDynamic, m.DimDynamic_t, rule=dynamic_fix_yd)
        m.dynamic_con2 = pyo.Constraint(m.DimDynamic, rule=dynamic_fix_yd_con2)
                
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
     
    # set objective 
    if obj == "A":
        m.Obj = pyo.Objective(rule=ComputeTrace, sense=pyo.maximize)    
    elif obj =="D":
        
        def _model_i(b):
            build_model_external(b)
        m.my_block = pyo.Block(rule=_model_i)
        
        for i in m.DimFIM:
            for j in range(i, len(FIMSet[0])):
                def eq_fim(m):
                    return m.TotalFIM[i,j] == m.my_block.egb.inputs['ele_'+str(i)+'_'+str(j)]
                
                con_name = "con"+str(i)+str(j)
                m.add_component(con_name, pyo.Constraint(expr=eq_fim))

        # add objective to maximize log det
        m.Obj = pyo.Objective(expr=m.my_block.egb.outputs['log_det'], sense=pyo.maximize)

    
    return m 

In [12]:
if strategy3: 
    num_fixed_opt = 11 
    
else:
    num_fixed_opt = 9

num_dynamic_opt = 5

mip_option = False
objective = "D"
sparse_opt = True
fix_opt = False

each_manual_num = 5
total_manual_num = 20
budget_opt = 10000  

#discretize_time = 10
discretize_time = 30
#num_dynamic_time = np.linspace(2,220,dynamic_Nt)
num_dynamic_time = np.linspace(10,220,dynamic_Nt)
    
print(num_dynamic_time)

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 
    
print(dynamic_time_dict)

[ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130. 140.
 150. 160. 170. 180. 190. 200. 210. 220.]
{0: 10.0, 1: 20.0, 2: 30.0, 3: 40.0, 4: 50.0, 5: 60.0, 6: 70.0, 7: 80.0, 8: 90.0, 9: 100.0, 10: 110.0, 11: 120.0, 12: 130.0, 13: 140.0, 14: 150.0, 15: 160.0, 16: 170.0, 17: 180.0, 18: 190.0, 19: 200.0, 20: 210.0, 21: 220.0}


In [13]:
mod = compute_FIM(num_total, calculator.fim_collection, cost, mixed_integer=mip_option, 
                  num_fixed = num_fixed_opt, num_dynamic = num_dynamic_opt, 
                  num_dynamic_t = dynamic_Nt, num_dynamic_t_name = num_dynamic_time,
                  each_manual_number=each_manual_num, manual_number=total_manual_num,
                  dynamic_install_cost = install_cost_dynamic, discretize_time = discretize_time, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, budget=budget_opt)

if not mip_option and objective=="A":
    solver = pyo.SolverFactory('ipopt')
    solver.options['linear_solver'] = "ma57"
    solver.solve(mod, tee=True)

elif mip_option and objective=="A":
    #solver = pyo.SolverFactory('mindtpy')
    #results = solver.solve(mod, mip_solver='gurobi_persistent', nlp_solver='ipopt', 
    #                       time_limit=36000, iteration_limit=100000,
    #                       tee=True, integer_tolerance=0.5)
    solver = pyo.SolverFactory('gurobi', solver_io="python")
    solver.options['mipgap'] = 0.1
    solver.options['NumericFocus'] = 2
    results = solver.solve(mod, tee=True)
    
elif objective=="D":  
    solver = pyo.SolverFactory('cyipopt')
    solver.config.options['hessian_approximation'] = 'limited-memory' 
    additional_options={'max_iter':3000}
    
    for k,v in additional_options.items():
        solver.config.options[k] = v
    results = solver.solve(mod, tee=True)


 Consider M =
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
   logdet =  0.0 

Eigvals: [1. 1. 1. 1. 1.]

 Consider M =
 [[ 0.99947596  0.00791358 -0.01247945 -0.01375091 -0.00114872]
 [ 0.00791358  0.99443435 -0.00514087 -0.01940253  0.00306218]
 [-0.01247945 -0.00514087  1.0232093   0.01584668 -0.00196199]
 [-0.01375091 -0.01940253  0.01584668  0.99173097 -0.00242128]
 [-0.00114872  0.00306218 -0.00196199 -0.00242128  1.02707272]]
[0.999475963862527, 0.00791357741242674, -0.012479445352052693, -0.013750906139687508, -0.0011487196079140215, 0.9944343532595269, -0.005140873019855163, -0.019402525695922886, 0.0030621781958508816, 1.0232092997889697, 0.015846679209539912, -0.0019619908770899834, 0.9917309733993959, -0.0024212814373732582, 1.0270727226132383]
   logdet =  0.03418040961519511 

Eigvals: [0.97166813 1.04339059 0.98951605 1.00453344 1.02681511]

 Conside


 Consider M =
 [[ 21.21380668  -0.50841068 -11.96240132  -3.12424101  -5.03107211]
 [ -0.50841068   0.38605518   0.70353713   0.85007909   0.04326757]
 [-11.96240132   0.70353713  19.58295453  -0.81372313  -8.19986402]
 [ -3.12424101   0.85007909  -0.81372313  25.59627453  -1.69466236]
 [ -5.03107211   0.04326757  -8.19986402  -1.69466236  16.85828043]]
[21.213806678839088, -0.5084106847284705, -11.962401324777096, -3.1242410098656372, -5.031072113925966, 0.3860551757637694, 0.7035371275534855, 0.8500790940733608, 0.04326756598759769, 19.582954533381876, -0.8137231348465231, -8.19986401865183, 25.596274532596667, -1.694662361197303, 16.858280433445387]
   logdet =  9.251277927714227 

Eigvals: [ 1.90385764  0.28418559 33.17385029 22.65286592 25.62261191]

 Consider M =
 [[ 21.83076841  -0.48584594 -11.82486653  -0.18824748  -6.78782542]
 [ -0.48584594   0.3969456    0.65347315   1.09545839  -0.03903411]
 [-11.82486653   0.65347315  20.02733784  -4.69702495  -8.36566703]
 [ -0.18824748


 Consider M =
 [[ 6.32183514e+02  2.72454153e+00 -4.38653169e+02  1.34919546e+03
  -9.57978442e+01]
 [ 2.72454153e+00  7.19108570e-01 -1.31319944e+00  1.40980357e+01
  -2.06900340e+00]
 [-4.38653169e+02 -1.31319944e+00  3.37196614e+02 -9.75879118e+02
   4.03480247e+01]
 [ 1.34919546e+03  1.40980357e+01 -9.75879118e+02  3.42709814e+03
  -2.03281244e+02]
 [-9.57978442e+01 -2.06900340e+00  4.03480247e+01 -2.03281244e+02
   5.62680436e+01]]
[632.1835138225298, 2.7245415283731105, -438.6531686225578, 1349.195460849421, -95.79784417838972, 0.7191085700316183, -1.313199444557282, 14.098035693494184, -2.069003397802209, 337.19661412818357, -975.879117587484, 40.34802468712848, 3427.0981444551585, -203.28124367220335, 56.268043552331136]
   logdet =  18.438362977986177 

Eigvals: [4.27556313e+03 1.11463342e+02 5.88439348e+01 7.08255145e+00
 5.12462947e-01]

 Consider M =
 [[ 7.17864202e+02  2.66381207e+00 -5.28882669e+02  1.58823791e+03
  -8.01067442e+01]
 [ 2.66381207e+00  7.26098727e-01 -1.2


 Consider M =
 [[ 1.04420749e+03  3.29922848e+00 -7.61698982e+02  2.39617062e+03
  -1.38194533e+02]
 [ 3.29922848e+00  7.30221174e-01 -1.53612556e+00  1.57975526e+01
  -2.53570787e+00]
 [-7.61698982e+02 -1.53612556e+00  5.93246477e+02 -1.80120489e+03
   7.21394961e+01]
 [ 2.39617062e+03  1.57975526e+01 -1.80120489e+03  6.19909862e+03
  -2.97290496e+02]
 [-1.38194533e+02 -2.53570787e+00  7.21394961e+01 -2.97290496e+02
   6.83189553e+01]]
[1044.2074911606562, 3.2992284776125778, -761.6989817576131, 2396.170623895935, -138.19453299715252, 0.7302211737294669, -1.53612556367566, 15.797552614333494, -2.535707872179404, 593.2464770107761, -1801.2048895430037, 72.13949607932764, 6199.098617230024, -297.2904962320929, 68.31895533712242]
   logdet =  19.65855941564006 

Eigvals: [7.69491418e+03 1.32974334e+02 6.73860676e+01 9.81779711e+00
 5.09387290e-01]

 Consider M =
 [[ 1.04761700e+03  3.16874328e+00 -7.71226533e+02  2.41851258e+03
  -1.34149051e+02]
 [ 3.16874328e+00  7.27029277e-01 -1.383


 Consider M =
 [[ 1.10777392e+03  2.52447945e+00 -8.16087302e+02  2.57952365e+03
  -1.51382082e+02]
 [ 2.52447945e+00  7.01587265e-01 -7.53498146e-01  1.36728729e+01
  -2.68859567e+00]
 [-8.16087302e+02 -7.53498146e-01  6.42089894e+02 -1.96299561e+03
   8.00652722e+01]
 [ 2.57952365e+03  1.36728729e+01 -1.96299561e+03  6.76933719e+03
  -3.20599968e+02]
 [-1.51382082e+02 -2.68859567e+00  8.00652722e+01 -3.20599968e+02
   7.66561724e+01]]
[1107.77391614033, 2.524479452272807, -816.0873015657883, 2579.523654740677, -151.3820820190993, 0.7015872654764739, -0.7534981463451966, 13.672872871621765, -2.6885956699874263, 642.0898940253176, -1962.9956060413986, 80.06527216162553, 6769.337192647568, -320.59996816688295, 76.65617241961698]
   logdet =  19.935302361952328 

Eigvals: [8.36987915e+03 1.41379342e+02 7.39739258e+01 1.08474072e+01
 4.78941043e-01]

 Consider M =
 [[ 1.10818954e+03  2.51463449e+00 -8.16474066e+02  2.58081567e+03
  -1.51496655e+02]
 [ 2.51463449e+00  7.02340748e-01 -7.45

...


 Consider M =
 [[ 1.12220198e+03  2.76576701e+00 -8.24725787e+02  2.62948249e+03
  -1.55659398e+02]
 [ 2.76576701e+00  7.34964286e-01 -8.02118404e-01  1.44541095e+01
  -2.98106652e+00]
 [-8.24725787e+02 -8.02118404e-01  6.48992628e+02 -1.99631249e+03
   8.09735398e+01]
 [ 2.62948249e+03  1.44541095e+01 -1.99631249e+03  6.93332558e+03
  -3.32248892e+02]
 [-1.55659398e+02 -2.98106652e+00  8.09735398e+01 -3.32248892e+02
   8.20763446e+01]]
[1122.2019794867629, 2.76576701098007, -824.725787033626, 2629.4824943210065, -155.65939806691802, 0.7349642863177327, -0.8021184036622304, 14.45410950054989, -2.981066520537787, 648.992627955861, -1996.312487904028, 80.97353984227038, 6933.325584488546, -332.24889188087457, 82.0763445958167]
   logdet =  20.120550647629972 

Eigvals: [8.55416747e+03 1.41688195e+02 7.94497969e+01 1.15332203e+01
 4.92821137e-01]

 Consider M =
 [[ 1.12218546e+03  2.76553930e+00 -8.24725671e+02  2.62945447e+03
  -1.55650813e+02]
 [ 2.76553930e+00  7.34993917e-01 -8.02517


 Consider M =
 [[ 1.12219760e+03  2.76554245e+00 -8.24725379e+02  2.62948158e+03
  -1.55658551e+02]
 [ 2.76554245e+00  7.34957417e-01 -8.01977717e-01  1.44537286e+01
  -2.98107393e+00]
 [-8.24725379e+02 -8.01977717e-01  6.48994358e+02 -1.99631897e+03
   8.09726676e+01]
 [ 2.62948158e+03  1.44537286e+01 -1.99631897e+03  6.93335759e+03
  -3.32246083e+02]
 [-1.55658551e+02 -2.98107393e+00  8.09726676e+01 -3.32246083e+02
   8.20777919e+01]]
[1122.1976026790774, 2.7655424537225444, -824.7253793767671, 2629.4815840672204, -155.65855108556403, 0.7349574172807172, -0.8019777170040706, 14.453728592275413, -2.9810739343608006, 648.9943575543059, -1996.3189708642, 80.97266759921101, 6933.357594268021, -332.2460833459103, 82.0777919137844]
   logdet =  20.120562437289855 

Eigvals: [8.55419506e+03 1.41689963e+02 7.94513125e+01 1.15331546e+01
 4.92812609e-01]

 Consider M =
 [[ 1.12220620e+03  2.76468810e+00 -8.24719890e+02  2.62952196e+03
  -1.55672955e+02]
 [ 2.76468810e+00  7.34827570e-01 -7.99


 Consider M =
 [[ 1.12220483e+03  2.76527551e+00 -8.24723772e+02  2.62949969e+03
  -1.55666240e+02]
 [ 2.76527551e+00  7.34912392e-01 -8.01275519e-01  1.44523561e+01
  -2.98132483e+00]
 [-8.24723772e+02 -8.01275519e-01  6.48993067e+02 -1.99631810e+03
   8.09755596e+01]
 [ 2.62949969e+03  1.44523561e+01 -1.99631810e+03  6.93341061e+03
  -3.32266844e+02]
 [-1.55666240e+02 -2.98132483e+00  8.09755596e+01 -3.32266844e+02
   8.20787151e+01]]
[1122.2048288875544, 2.765275511283852, -824.7237718796371, 2629.4996903738356, -155.66624044502882, 0.7349123921568147, -0.8012755190645409, 14.452356057059426, -2.981324829007129, 648.9930666536092, -1996.3181037076477, 80.97555963082776, 6933.41060954315, -332.26684356725616, 82.07871512992726]
   logdet =  20.1205624301308 

Eigvals: [8.55425122e+03 1.41690570e+02 7.94535631e+01 1.15340262e+01
 4.92756063e-01]

 Consider M =
 [[ 1.12220483e+03  2.76527551e+00 -8.24723772e+02  2.62949969e+03
  -1.55666240e+02]
 [ 2.76527551e+00  7.34912392e-01 -8.01


 Consider M =
 [[ 1.12219966e+03  2.76557280e+00 -8.24725899e+02  2.62948495e+03
  -1.55659127e+02]
 [ 2.76557280e+00  7.34956230e-01 -8.01992420e-01  1.44536634e+01
  -2.98105513e+00]
 [-8.24725899e+02 -8.01992420e-01  6.48994072e+02 -1.99631939e+03
   8.09732554e+01]
 [ 2.62948495e+03  1.44536634e+01 -1.99631939e+03  6.93335868e+03
  -3.32247224e+02]
 [-1.55659127e+02 -2.98105513e+00  8.09732554e+01 -3.32247224e+02
   8.20773513e+01]]
[1122.1996579346971, 2.765572798565986, -824.725898591519, 2629.4849482243008, -155.65912719301332, 0.7349562298236623, -0.801992420187854, 14.453663438833242, -2.9810551341974048, 648.9940722905711, -1996.3193890122234, 80.97325544798433, 6933.358676415425, -332.2472243706303, 82.07735129869373]
   logdet =  20.120562455568493 

Eigvals: [8.55419868e+03 1.41689315e+02 7.94506481e+01 1.15332537e+01
 4.92814550e-01]

 Consider M =
 [[ 1.12220001e+03  2.76562469e+00 -8.24725678e+02  2.62948430e+03
  -1.55659377e+02]
 [ 2.76562469e+00  7.34959115e-01 -8.0


 Consider M =
 [[ 1.12219967e+03  2.76562709e+00 -8.24726037e+02  2.62948380e+03
  -1.55658773e+02]
 [ 2.76562709e+00  7.34961264e-01 -8.02072776e-01  1.44538453e+01
  -2.98104596e+00]
 [-8.24726037e+02 -8.02072776e-01  6.48993823e+02 -1.99631815e+03
   8.09732032e+01]
 [ 2.62948380e+03  1.44538453e+01 -1.99631815e+03  6.93335058e+03
  -3.32246572e+02]
 [-1.55658773e+02 -2.98104596e+00  8.09732032e+01 -3.32246572e+02
   8.20772620e+01]]
[1122.19967498159, 2.7656270915840184, -824.7260373819081, 2629.483796856627, -155.6587725303203, 0.7349612636115578, -0.8020727760288254, 14.453845271024907, -2.9810459644785947, 648.9938227608809, -1996.3181534039654, 80.9732031895273, 6933.350576879868, -332.2465721601394, 82.0772620028657]
   logdet =  20.120562367537396 

Eigvals: [8.55419078e+03 1.41689079e+02 7.94504338e+01 1.15331807e+01
 4.92820233e-01]

 Consider M =
 [[ 1.12221031e+03  2.76400011e+00 -8.24715980e+02  2.62953400e+03
  -1.55683620e+02]
 [ 2.76400011e+00  7.34767476e-01 -7.9877

In [14]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 1.12219999e+03  2.76560503e+00 -8.24725801e+02  2.62948478e+03
  -1.55659328e+02]
 [ 2.76560503e+00  7.34957864e-01 -8.02011119e-01  1.44537530e+01
  -2.98107211e+00]
 [-8.24725801e+02 -8.02011119e-01  6.48993748e+02 -1.99631795e+03
   8.09733265e+01]
 [ 2.62948478e+03  1.44537530e+01 -1.99631795e+03  6.93335459e+03
  -3.32248146e+02]
 [-1.55659328e+02 -2.98107211e+00  8.09733265e+01 -3.32248146e+02
   8.20774044e+01]]
trace: 8787.360697555365
det: 547329992.5757369
[8.55419468e+03 1.41689210e+02 7.94507506e+01 1.15332400e+01
 4.92815099e-01]
Pyomo OF: 20.120562455911998
Log_det: 20.1205624555915


In [15]:
mod.display()

Model unknown

  Variables:
    cov_y : Size=7381, Index=NumRes_half
        Key        : Lower : Value                  : Upper : Fixed : Stale : Domain
            (0, 0) : 1e-06 :     0.4874060193310004 :     1 : False : False : NonNegativeReals
            (0, 1) : 1e-06 :    0.48740602849804976 :     1 : False : False : NonNegativeReals
            (0, 2) : 1e-06 :  9.901704786849633e-07 :     1 : False : False : NonNegativeReals
            (0, 3) : 1e-06 :   9.90042250948811e-07 :     1 : False : False : NonNegativeReals
            (0, 4) : 1e-06 :     0.4874060246173164 :     1 : False : False : NonNegativeReals
            (0, 5) : 1e-06 :      0.487406009173574 :     1 : False : False : NonNegativeReals
            (0, 6) : 1e-06 :    0.48740601944745293 :     1 : False : False : NonNegativeReals
            (0, 7) : 1e-06 :    0.12022186885069774 :     1 : False : False : NonNegativeReals
            (0, 8) : 1e-06 :  9.900562387604038e-07 :     1 : False : False : NonNegat

          (14, 61) : 1e-06 : 1.0619824119550593e-06 :     1 : False : False : NonNegativeReals
          (14, 62) : 1e-06 : 1.0581248113173294e-06 :     1 : False : False : NonNegativeReals
          (14, 63) : 1e-06 :  1.046141978563763e-06 :     1 : False : False : NonNegativeReals
          (14, 64) : 1e-06 : 1.1323120312210939e-06 :     1 : False : False : NonNegativeReals
          (14, 65) : 1e-06 : 1.0586141938075178e-06 :     1 : False : False : NonNegativeReals
          (14, 66) : 1e-06 : 1.0465305619339176e-06 :     1 : False : False : NonNegativeReals
          (14, 67) : 1e-06 : 1.0619280518982604e-06 :     1 : False : False : NonNegativeReals
          (14, 68) : 1e-06 : 1.0752705549902641e-06 :     1 : False : False : NonNegativeReals
          (14, 69) : 1e-06 : 1.0861610433387853e-06 :     1 : False : False : NonNegativeReals
          (14, 70) : 1e-06 :  1.091449950032722e-06 :     1 : False : False : NonNegativeReals
          (14, 71) : 1e-06 :  1.071909504477988e-0

          (17, 22) : 1e-06 : 1.0795039888436184e-06 :     1 : False : False : NonNegativeReals
          (17, 23) : 1e-06 :  1.081972384487307e-06 :     1 : False : False : NonNegativeReals
          (17, 24) : 1e-06 : 1.0886371931627205e-06 :     1 : False : False : NonNegativeReals
          (17, 25) : 1e-06 : 1.0779716365905513e-06 :     1 : False : False : NonNegativeReals
          (17, 26) : 1e-06 : 1.0536678347274475e-06 :     1 : False : False : NonNegativeReals
          (17, 27) : 1e-06 : 1.0587941018186019e-06 :     1 : False : False : NonNegativeReals
          (17, 28) : 1e-06 : 1.0888292965261744e-06 :     1 : False : False : NonNegativeReals
          (17, 29) : 1e-06 : 1.0820247350512353e-06 :     1 : False : False : NonNegativeReals
          (17, 30) : 1e-06 :  1.060946264849531e-06 :     1 : False : False : NonNegativeReals
          (17, 31) : 1e-06 : 1.0792751551446526e-06 :     1 : False : False : NonNegativeReals
          (17, 32) : 1e-06 :  1.066790561313849e-0

          (32, 99) : 1e-06 :   1.06728196429803e-06 :     1 : False : False : NonNegativeReals
         (32, 100) : 1e-06 : 1.0696319642247771e-06 :     1 : False : False : NonNegativeReals
         (32, 101) : 1e-06 : 1.1080812944641503e-06 :     1 : False : False : NonNegativeReals
         (32, 102) : 1e-06 :  1.129604852745294e-06 :     1 : False : False : NonNegativeReals
         (32, 103) : 1e-06 :  1.109425199110822e-06 :     1 : False : False : NonNegativeReals
         (32, 104) : 1e-06 : 1.0798408253517133e-06 :     1 : False : False : NonNegativeReals
         (32, 105) : 1e-06 : 1.0648100421665775e-06 :     1 : False : False : NonNegativeReals
         (32, 106) : 1e-06 :  1.029009990593919e-06 :     1 : False : False : NonNegativeReals
         (32, 107) : 1e-06 : 1.0965169557475517e-06 :     1 : False : False : NonNegativeReals
         (32, 108) : 1e-06 :  1.143547689184014e-06 :     1 : False : False : NonNegativeReals
         (32, 109) : 1e-06 :  1.029202477579027e-0

          (35, 54) : 1e-06 : 1.0760718787590506e-06 :     1 : False : False : NonNegativeReals
          (35, 55) : 1e-06 : 1.0824618647362852e-06 :     1 : False : False : NonNegativeReals
          (35, 56) : 1e-06 :  1.079765507041407e-06 :     1 : False : False : NonNegativeReals
          (35, 57) : 1e-06 :  1.078654190392837e-06 :     1 : False : False : NonNegativeReals
          (35, 58) : 1e-06 : 1.0761144272243552e-06 :     1 : False : False : NonNegativeReals
          (35, 59) : 1e-06 : 1.0762597518339528e-06 :     1 : False : False : NonNegativeReals
          (35, 60) : 1e-06 :  1.077098991476733e-06 :     1 : False : False : NonNegativeReals
          (35, 61) : 1e-06 : 1.0795441785532347e-06 :     1 : False : False : NonNegativeReals
          (35, 62) : 1e-06 :  1.072971061535868e-06 :     1 : False : False : NonNegativeReals
          (35, 63) : 1e-06 : 1.0547154247146318e-06 :     1 : False : False : NonNegativeReals
          (35, 64) : 1e-06 :  2.356037122192958e-0

...

         (50, 115) :  None :  -4.637004318819414e-07 :   0.0
         (50, 116) :  None :  -2.639853922755308e-07 :   0.0
         (50, 117) :  None :  -5.316632695505324e-07 :   0.0
         (50, 118) :  None :  -4.650626334068531e-07 :   0.0
         (50, 119) :  None :  -4.846051351902663e-07 :   0.0
         (50, 120) :  None : -4.6165162655972597e-07 :   0.0
          (51, 52) :  None :     -0.9999955240787887 :   0.0
          (51, 53) :  None :     -0.9999955466166225 :   0.0
          (51, 54) :  None :     -0.9999957609896941 :   0.0
          (51, 55) :  None :     -0.9999957540867722 :   0.0
          (51, 56) :  None :     -0.9999957570046507 :   0.0
          (51, 57) :  None :     -0.9999957582051047 :   0.0
          (51, 58) :  None :     -0.9999957609438674 :   0.0
          (51, 59) :  None :     -0.9999957607873323 :   0.0
          (51, 60) :  None :      -0.999995759882935 :   0.0
          (51, 61) :  None :      -0.999995757243832 :   0.0
          (51, 62) :  No

        Key        : Lower : Body                    : Upper
            (0, 1) :  None :   9.918282972698478e-09 :   0.0
            (0, 2) :  None :     -0.5125939905129515 :   0.0
            (0, 3) :  None :    -0.20119487035508224 :   0.0
            (0, 4) :  None :    -0.36360137750467114 :   0.0
            (0, 5) :  None :     -0.3636013929442219 :   0.0
            (0, 6) :  None :     -0.5125939905403034 :   0.0
            (0, 7) :  None :  -0.0006208347924312602 :   0.0
            (0, 8) :  None :     -0.5125939905609171 :   0.0
            (0, 9) :  None :     -0.5125939906088046 :   0.0
           (0, 10) :  None :   9.877953122128247e-09 :   0.0
           (0, 11) :  None :  -8.976601298107012e-08 :   0.0
           (0, 12) :  None :  -8.514017589682809e-08 :   0.0
           (0, 13) :  None : -1.0238676686213758e-07 :   0.0
           (0, 14) :  None : -1.2787916157836742e-07 :   0.0
           (0, 15) :  None : -1.2231330010298612e-07 :   0.0
           (0, 16) :  No

          (48, 94) :  None :  -9.916771527034955e-08 :   0.0
          (48, 95) :  None :  -1.383086303330493e-07 :   0.0
          (48, 96) :  None : -1.7293278047726577e-07 :   0.0
          (48, 97) :  None : -1.6274316948775285e-07 :   0.0
          (48, 98) :  None : -1.3895833117362715e-07 :   0.0
          (48, 99) :  None :  -8.663713983644661e-08 :   0.0
         (48, 100) :  None :  -9.088929385342515e-08 :   0.0
         (48, 101) :  None :  -2.129786589825038e-07 :   0.0
         (48, 102) :  None :  -5.804183931978818e-07 :   0.0
         (48, 103) :  None :  -2.215201036886722e-07 :   0.0
         (48, 104) :  None : -1.1162779867626322e-07 :   0.0
         (48, 105) :  None :  -8.233261040845979e-08 :   0.0
         (48, 106) :  None : -3.2192132329751756e-08 :   0.0
         (48, 107) :  None : -1.5880846617154352e-07 :   0.0
         (48, 108) :  None : -2.3728820867097156e-06 :   0.0
         (48, 109) :  None :  -3.242192275319589e-08 :   0.0
         (48, 110) :  No

        Key        : Lower : Body                    : Upper
            (0, 1) :  None :     -0.5125939905872825 :   0.0
            (0, 2) :  None :    9.84395191164417e-09 :   0.0
            (0, 3) :  None :     -0.3113991103139173 :   0.0
            (0, 4) :  None :     -0.1489926031643285 :   0.0
            (0, 5) :  None :    -0.14899258772477764 :   0.0
            (0, 6) :  None :   9.871303829900313e-09 :   0.0
            (0, 7) :  None :     -0.5119731458765683 :   0.0
            (0, 8) :  None :    9.89191751683781e-09 :   0.0
            (0, 9) :  None :   9.939804923586593e-09 :   0.0
           (0, 10) :  None :     -0.5125939905469528 :   0.0
           (0, 11) :  None :     -0.5125938909029866 :   0.0
           (0, 12) :  None :     -0.5125938955288236 :   0.0
           (0, 13) :  None :     -0.5125938782822328 :   0.0
           (0, 14) :  None :      -0.512593852789838 :   0.0
           (0, 15) :  None :     -0.5125938583556995 :   0.0
           (0, 16) :  No

          (54, 64) :  None :  -2.179266039871595e-06 :   0.0
          (54, 65) :  None :     -0.9999987406371048 :   0.0
          (54, 66) :  None :     -0.9999987712487453 :   0.0
          (54, 67) :  None :     -0.9999987307301902 :   0.0
          (54, 68) :  None :     -0.9999986812310102 :   0.0
          (54, 69) :  None :     -0.9999986226764412 :   0.0
          (54, 70) :  None :     -0.9999985838645613 :   0.0
          (54, 71) :  None :     -0.9999986954578444 :   0.0
          (54, 72) :  None :     -0.9999986919527921 :   0.0
          (54, 73) :  None :     -0.9999986186141531 :   0.0
          (54, 74) :  None :      -0.999998572641603 :   0.0
          (54, 75) :  None :     -0.9999986068153354 :   0.0
          (54, 76) :  None :     -0.9999986514569241 :   0.0
          (54, 77) :  None :     -0.9999987262333743 :   0.0
          (54, 78) :  None :     -0.9999987298063099 :   0.0
          (54, 79) :  None :     -0.9999987314745254 :   0.0
          (54, 80) :  No

In [16]:
# problem size calculator
if True:
    nr = num_total
    npa = 5
    nd = 5
    nd_time = 110
    num_var = (nr*nr+nr)/2 + (npa*npa+npa)/2 + nd +2
    num_equal = (npa*npa+npa)/2+2
    num_ineq = 1.5*(nr**2-nr)+2+2*nd+2*nd*nd_time

    print("# variables:", num_var)
    print("# equality constraints:", num_equal)
    print("# inequality constraints:", num_ineq)

# variables: 7403.0
# equality constraints: 17.0
# inequality constraints: 22892.0


In [17]:
ans_y = np.zeros((num_total,num_total))

pyomo_y = np.zeros((num_total, num_total))

for i in range(num_total):
    for j in range(i, num_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = ans_y[j,i] = cov 
        
print("Cost for this stragy:", pyo.value(mod.cost))
print("Cost if no budget:", sum(cost))



#print(pyo.value(mod.cov_y[9,9]))
#print(pyo.value(mod.if_install_dynamic[9]))

#for i in range(9,119):
#    print(pyo.value(mod.cov_y[i,i]))
#print(sum(pyo.value(mod.cov_y[i,i]) for i in range(9,119)))

Cost for this stragy: 10000.000099912904
Cost if no budget: 33000


In [18]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

[[0.49 0.49 0.   ... 0.   0.   0.  ]
 [0.49 0.49 0.   ... 0.   0.   0.  ]
 [0.   0.   0.51 ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [19]:
import pickle

store = False

if store:
    file = open('Mar20_A_110_8000_strategy3_mip', 'wb')

    pickle.dump(ans_y, file)

    file.close()

In [20]:
for c in range(num_static):
    print(static_name[c], ": ", ans_y[c,c])

sol_y = np.asarray([ans_y[i,i] for i in range(num_static, num_total)])

sol_y = np.reshape(sol_y, (num_dynamic, dynamic_Nt))
np.around(sol_y)

for r in range(len(sol_y)):
    #print(dynamic_name[r], ": ", sol_y[r])
    print(dynamic_name[r], "(time points [min]):")
    for i, t in enumerate(sol_y[r]):
        if t>0.5:
            print(dynamic_time_dict[i])
    
# verify cost 
real_cost = 0 
for i in range(num_total):
    real_cost += ans_y[i,i]*cost[i]
    
dynamic_inst = [0]*num_dynamic
for j in range(num_dynamic):
    for t in range(dynamic_Nt):
        if sol_y[j,t]>0.99:
            dynamic_inst[j] = 1 
            
real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
print(dynamic_inst)
print(real_cost)
    


Ads.gas_inlet.F :  0.49
Ads.gas_outlet.F :  0.49
Ads.gas_outlet.T :  0.51
Ads.gas_outlet.z("CO2") :  0.2
Des.gas_inlet.F :  0.85
Des.gas_outlet.F :  0.85
Des.gas_outlet.T :  1.0
Des.gas_outlet.z("CO2") :  0.12
Ads.T_g.Value(19,10) :  0.51
Ads.T_g.Value(23,10) :  0.54
Ads.T_g.Value(28,10) :  0.46
Ads.gas_outlet.z("CO2") (time points [min]):
110.0
140.0
180.0
Des.gas_outlet.z("CO2") (time points [min]):
20.0
160.0
190.0
Ads.z("CO2",19,10) (time points [min]):
100.0
Ads.z("CO2",23,10) (time points [min]):
100.0
Ads.z("CO2",28,10) (time points [min]):
100.0
130.0
180.0
[1, 1, 1, 1, 1]
9985.0


In [21]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [22]:
#measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
#measure_name = ['Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
measure_name = ['a', 'b', 'c', 'd', 'e']

#time_set = ['500', '1000', '1500', '2000', '2200']
time_set = np.linspace(20, 2200, number_t)
time_set = [str(i) for i in time_set]
print(len(time_set), len(measure_name))
print(np.shape(sol_y))

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, time_set, measure_name, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

NameError: name 'number_t' is not defined

In [ ]:
def plot_data(budget, a_cri, d_cri):

    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    #plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(A-optimality)', fontweight="bold")
    
    plt.show()
    
    
    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(D-optimality)', fontweight="bold")
    
    plt.show()

In [ ]:
a_set = [65693.94, 66574.63, 67474.485, 67608.817]
d_set = [np.exp(30.13), np.exp(30.719), np.exp(31.1205), np.exp(31.2586)]

plot_data([3000,4000,5000,6400], a_set, d_set)